In [1]:
import tensorflow as tf
from tensorflow import keras
from collections import Counter
from component.respiratory_tract import get_respiratory_tract_model
from utils.feature_extraction import  get_MFCCS_v2, oversample, undersample
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, classification_report
import numpy as np
import glob
import tqdm
import os

In [7]:
def make_data(folder, n_mfcc, num_segment, segment_len, use_oversample=False, use_undersample=False):
    X1 = glob.glob(os.path.join(folder, 'p', '*.wav'))
    X0 = glob.glob(os.path.join(folder, 'n', '*.wav'))

    y = []
    X = []
    for file in tqdm.tqdm(X1):
        try:
            X.append(get_MFCCS_v2(file, n_mfccs=n_mfcc, num_segments=num_segment, segment_length=segment_len))
            y.append(1)
        except:
            pass
    for file in tqdm.tqdm(X0):
        try:
            X.append(get_MFCCS_v2(file, n_mfccs=n_mfcc, num_segments=num_segment, segment_length=segment_len))
            y.append(0)
        except:
            pass
    X = np.array(X)
    y = np.array(y)
    if use_oversample:
        X, y = oversample(X, y)
    elif use_undersample:
        X, y = undersample(X, y)
    return X, y

In [8]:
import random
class DataGenerator(keras.utils.Sequence):
    def __init__(self, X, y, batch_size):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.indexes = np.arange(len(X))

    def __len__(self):
        return len(self.indexes)

    def __getitem__(self, _):
        indexes = np.random.choice(self.indexes, self.batch_size, replace=False)
        return self.X[indexes], self.y[indexes]

In [9]:
def get_model(n_mfcc, num_segment, weight_path=None):
    model = get_respiratory_tract_model(input_shape=(n_mfcc*3+3, num_segment, 1))
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['acc'])
    if weight_path is not None:
        model.load_weights(weight_path)
    return model

def train_and_test(model_name, train_folder, test_folder, n_mfcc, num_segment, segment_len, clear_cache=True, use_oversample=True, use_undersample=False):
    if clear_cache:
        get_MFCCS_v2.cache_clear()
        cached_data = glob.glob("cache/*.npy")
        for file in cached_data:
            os.remove(file)

    #build data
    X, y = make_data(train_folder, n_mfcc, num_segment, segment_len, use_oversample=use_oversample, use_undersample=use_undersample)
    X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8, random_state=69, stratify=y)
    class_weights = class_weight.compute_class_weight('balanced', np.unique(y), y)
    class_weights = {1 - i: x for i,x in enumerate(class_weights)}
    print(f"Class weights: {class_weights}")
    print(f"Label counts: {Counter(y)}")
    train_generator = DataGenerator(X_train, y_train, 8)
    val_generator = DataGenerator(X_val, y_val, 8)

    #train
    if not os.path.exists(f'logs/{model_name}'):
        os.mkdir(f'logs/{model_name}')
    model = get_model(n_mfcc, num_segment)
    history = model.fit(train_generator, steps_per_epoch=20, validation_steps=50, epochs=1000, validation_data=val_generator, class_weight=class_weights,
                        verbose=1, callbacks=[tf.keras.callbacks.ModelCheckpoint(f'pretrained_weight/{model_name}.h5', monitor="val_loss", save_weights_only=True, verbose=1, mode='min', save_best_only=True),
                             tf.keras.callbacks.TensorBoard(log_dir=f'logs/{model_name}')])

    #test
    X_test, y_test = make_data(test_folder,
                     n_mfcc, num_segment, segment_len)
    model.load_weights(f"pretrained_weight/{model_name}.h5")
    y_pred = model.predict(X_test)
    y_pred_r = np.rint(y_pred)
    return history, classification_report(y_pred_r, y_test), roc_auc_score(y_test, y_pred)

In [ ]:
# best param: 39 - 50 - 1024
n_mfccs = 13*np.array([2, 3, 4, 5], dtype=int) #M
num_segments = 10*np.array([3, 5, 7, 10, 12], dtype=int) #S
segment_length = np.exp2([8, 9, 10]).astype(int) #F
train_folder = 'coswara_wavs/train'
test_folder = 'coswara_wavs/test'
result = {}
for _ in range(15):
    while True:
        n_mfcc = random.choice(n_mfccs)
        num_segment = random.choice(num_segments)
        segment_len = random.choice(segment_length)
        if (n_mfcc, num_segment, segment_len) not in result:
            break
    print(f"n_mfcc={n_mfcc}, num_segment={num_segment}, segment_len={segment_len}")
    model_name = f'model_paper_{n_mfcc}_{num_segment}_{segment_len}'
    try:
        history, report, auc_score = train_and_test(model_name, train_folder, test_folder, n_mfcc, num_segment, segment_len)
    except Exception as e:
        result[(n_mfcc, num_segment, segment_len)] = f'error -> {type(e).__name__}: {str(e)}'
    else:
        result[(n_mfcc, num_segment, segment_len)] = history, report, auc_score



In [21]:
train_folder = 'coughvid/wavs/train'
test_folder = 'coswara_wavs/test'
n_mfccs = 39
num_segments = 50
segment_length = 1024
model_name = f'model_paper_{n_mfccs}_{num_segments}_{segment_length}'
history, report, auc_score = train_and_test(model_name, train_folder, test_folder,
                                            n_mfccs, num_segments, segment_length, use_oversample=True, clear_cache=False)

  0%|          | 0/7198 [00:00<?, ?it/s]

cant remove silence, path=coughvid/wavs/train\p\s1x2Ze9iVZNf6oFZDAZ4S9x3Cky2_cough-heavy.wav, error -> AttributeError: 'int' object has no attribute 'samples'. Skipping....
cant remove silence, path=coughvid/wavs/train\p\uwgjET5xm8NHkfQt7LsUpV0xC1W2_cough-heavy.wav, error -> AttributeError: 'int' object has no attribute 'samples'. Skipping....
cant remove silence, path=coughvid/wavs/train\p\uwgjET5xm8NHkfQt7LsUpV0xC1W2_cough-shallow.wav, error -> AttributeError: 'int' object has no attribute 'samples'. Skipping....


  5%|▍         | 344/7198 [00:06<05:03, 22.56it/s] 

cant remove silence, path=coughvid/wavs/train\n\0M4YhLM7FwWO5IjCqhi7MlBhvJv2_cough-heavy.wav, error -> AttributeError: 'int' object has no attribute 'samples'. Skipping....
cant remove silence, path=coughvid/wavs/train\n\0M4YhLM7FwWO5IjCqhi7MlBhvJv2_cough-shallow.wav, error -> AttributeError: 'int' object has no attribute 'samples'. Skipping....
cant remove silence, path=coughvid/wavs/train\n\0mkAUAbpROMIFjKpdFxlUnYTV262_cough-heavy.wav, error -> AttributeError: 'int' object has no attribute 'samples'. Skipping....
cant remove silence, path=coughvid/wavs/train\n\0mkAUAbpROMIFjKpdFxlUnYTV262_cough-shallow.wav, error -> AttributeError: 'int' object has no attribute 'samples'. Skipping....


 13%|█▎        | 948/7198 [00:35<02:30, 41.62it/s]

...

Class weights: {1: 1.0, 0: 1.0}
Label counts: Counter({1: 7119, 0: 7119})
Epoch 1/1000
20/20 [==============================] - 10s 190ms/step - loss: 1.1994 - acc: 0.4829 - val_loss: 0.9884 - val_acc: 0.4950

 64%|██████▍   | 329/514 [00:00<00:00, 2836.21it/s]


Epoch 01000: val_loss did not improve from 0.16812
cant remove silence, path=coswara_wavs/test\p\A31HJzr8ryMTgwJihXSI9PCot4j1_cough-shallow.wav, error -> AttributeError: 'int' object has no attribute 'samples'. Skipping....
cant remove silence, path=coswara_wavs/test\p\Kbvj1H92OLaeQy0YWHkPGvaH0hw2_cough-shallow.wav, error -> AttributeError: 'int' object has no attribute 'samples'. Skipping....
cant remove silence, path=coswara_wavs/test\n\2XgDqzQkqLX9SbmSHfbqdRxSmRD2_cough-heavy.wav, error -> AttributeError: 'int' object has no attribute 'samples'. Skipping....
cant remove silence, path=coswara_wavs/test\n\2XgDqzQkqLX9SbmSHfbqdRxSmRD2_cough-shallow.wav, error -> AttributeError: 'int' object has no attribute 'samples'. Skipping....
cant remove silence, path=coswara_wavs/test\n\aU8pLZV1OUQJV0GKeeIlgWhYWeA2_cough-shallow.wav, error -> AttributeError: 'int' object has no attribute 'samples'. Skipping....
cant remove silence, path=coswara_wavs/test\n\c18b81Qa5YY2RbEzblDNxMNQE312_cough-heav

100%|██████████| 514/514 [00:00<00:00, 3337.66it/s]



cant remove silence, path=coswara_wavs/test\n\Vq1h51z5x3Wp4wS2pCm6yEAZvu82_cough-heavy.wav, error -> AttributeError: 'int' object has no attribute 'samples'. Skipping....
cant remove silence, path=coswara_wavs/test\n\Vq1h51z5x3Wp4wS2pCm6yEAZvu82_cough-shallow.wav, error -> AttributeError: 'int' object has no attribute 'samples'. Skipping....


In [15]:
print(report)
"""
Train: CoughVid + undersample
Test: Coswara
              precision    recall  f1-score   support

         0.0       0.35      0.86      0.50       202
         1.0       0.60      0.12      0.20       362

    accuracy                           0.38       564
   macro avg       0.47      0.49      0.35       564
weighted avg       0.51      0.38      0.31       564
"""

              precision    recall  f1-score   support

         0.0       0.35      0.86      0.50       202
         1.0       0.60      0.12      0.20       362

    accuracy                           0.38       564
   macro avg       0.47      0.49      0.35       564
weighted avg       0.51      0.38      0.31       564



In [19]:
print(report)
"""
Train: CoughVid + Coswara positive + undersample
Test: Coswara
              precision    recall  f1-score   support

         0.0       0.57      0.88      0.69       317
         1.0       0.49      0.14      0.22       247

    accuracy                           0.56       564
   macro avg       0.53      0.51      0.46       564
weighted avg       0.53      0.56      0.49       564
"""

              precision    recall  f1-score   support

         0.0       0.57      0.88      0.69       317
         1.0       0.49      0.14      0.22       247

    accuracy                           0.56       564
   macro avg       0.53      0.51      0.46       564
weighted avg       0.53      0.56      0.49       564



In [22]:
print(report)



              precision    recall  f1-score   support

         0.0       0.91      0.88      0.89       507
         1.0       0.17      0.21      0.19        57

    accuracy                           0.81       564
   macro avg       0.54      0.55      0.54       564
weighted avg       0.83      0.81      0.82       564

